<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [23]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [24]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [25]:
!pip install 'keras<3.0.0' mediapipe-model-maker

Import the required packages.

In [26]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

In [27]:
import zipfile
from google.colab import drive

drive.mount('/content/gdrive/')
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/cup3.zip", 'r')
!unzip "/content/gdrive/MyDrive/cup3.zip"
train_dataset_path = "cup3/train"
validation_dataset_path = "cup3/validate"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
Archive:  /content/gdrive/MyDrive/cup3.zip
replace cup3/train/Annotations/a (10).xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: cup3/train/Annotations/a (10).xml  
replace cup3/train/Annotations/a (100).xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: cup3/train/Annotations/a (100).xml  
replace cup3/train/Annotations/a (102).xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: cup3/train/Annotations/a (102).xml  
  inflating: cup3/train/Annotations/a (103).xml  
  inflating: cup3/train/Annotations/a (104).xml  
  inflating: cup3/train/Annotations/a (105).xml  
  inflating: cup3/train/Annotations/a (106).xml  
  inflating: cup3/train/Annotations/a (107).xml  
  inflating: cup3/train/Annotations/a (108).xml  
  inflating: cup3/train/Annotations/a (109).xml  
  inflating: cup3/train/Annotations/a (11).xml  
  inflating: cup3/train/A

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [28]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    'cup3/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    'cup3/validate',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [29]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=50, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i256
Model: "retina_net_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   149056    
                             , '4': (Non

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


23/23 [==============================] - 63s 490ms/step - total_loss: 8.0650 - cls_loss: 7.4730 - box_loss: 0.0107 - model_loss: 8.0101 - val_total_loss: 1.7012 - val_cls_loss: 1.1873 - val_box_loss: 0.0092 - val_model_loss: 1.6462
Epoch 2/50
23/23 [==============================] - 5s 209ms/step - total_loss: 1.4893 - cls_loss: 1.1670 - box_loss: 0.0053 - model_loss: 1.4343 - val_total_loss: 1.2575 - val_cls_loss: 1.0717 - val_box_loss: 0.0026 - val_model_loss: 1.2026
Epoch 3/50
23/23 [==============================] - 6s 261ms/step - total_loss: 1.1728 - cls_loss: 1.0078 - box_loss: 0.0022 - model_loss: 1.1179 - val_total_loss: 0.9576 - val_cls_loss: 0.8360 - val_box_loss: 0.0013 - val_model_loss: 0.9026
Epoch 4/50
23/23 [==============================] - 6s 240ms/step - total_loss: 0.8671 - cls_loss: 0.7386 - box_loss: 0.0015 - model_loss: 0.8122 - val_total_loss: 0.6626 - val_cls_loss: 0.5550 - val_box_loss: 0.0011 - val_model_loss: 0.6076
Epoch 5/50
23/23 [========================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [30]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

5/5 [==============================] - 2s 138ms/step - total_loss: 0.2603 - cls_loss: 0.1514 - box_loss: 0.0011 - model_loss: 0.2056
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.781
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.781
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.806
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.806
 Average Recall     (AR) @[ 

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [31]:
model.export_model('cups.tflite')
!ls exported_model
files.download('exported_model/cups.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


cups.tflite  float_ckpt.data-00000-of-00001  float_ckpt.index  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>